<a href="https://colab.research.google.com/github/levinaanora/Corona-Virus-Tweet-NLP/blob/main/Coronavirus_tweets_NLP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf

In [2]:
data = pd.read_csv("Corona_NLP_train.csv", header = 0)
data.tail()

,UserName,ScreenName,Location,TweetAt,OriginalTweet,Sentiment
41152,44951,89903,"Wellington City, New Zealand",14-04-2020,Airline pilots offering to stock supermarket s...,Neutral
41153,44952,89904,NaN,14-04-2020,Response to complaint not provided citing COVI...,Extremely Negative
41154,44953,89905,NaN,14-04-2020,You know itÂ’s getting tough when @KameronWild...,Positive
41155,44954,89906,NaN,14-04-2020,Is it wrong that the smell of hand sanitizer i...,Neutral
41156,44955,89907,i love you so much || he/him,14-04-2020,@TartiiCat Well new/used Rift S are going for ...,Negative


In [3]:
del data["UserName"], data["ScreenName"], data["Location"], data["TweetAt"]
data.head()

,OriginalTweet,Sentiment
0,@MeNyrbie @Phil_Gahan @Chrisitv https://t.co/i...,Neutral
1,advice Talk to your neighbours family to excha...,Positive
2,Coronavirus Australia: Woolworths to give elde...,Positive
3,My food stock is not the only one which is emp...,Positive
4,"Me, ready to go at supermarket during the #COV...",Extremely Negative


In [4]:
data["Sentiment"].value_counts()

Positive              11422
Negative               9917
Neutral                7713
Extremely Positive     6624
Extremely Negative     5481
Name: Sentiment, dtype: int64

In [5]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41157 entries, 0 to 41156
Data columns (total 2 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   OriginalTweet  41157 non-null  object
 1   Sentiment      41157 non-null  object
dtypes: object(2)
memory usage: 643.2+ KB


In [6]:
missing_data = pd.DataFrame({'total_missing': data.isnull().sum(), 'perc_missing': (data.isnull().sum()/41156)*100})
missing_data

,total_missing,perc_missing
OriginalTweet,0,0.0
Sentiment,0,0.0


In [7]:
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()
data["Sentiment"] = label_encoder.fit_transform(data["Sentiment"])
data["Sentiment"]

0        3
1        4
2        4
3        4
4        0
        ..
41152    3
41153    0
41154    4
41155    3
41156    2
Name: Sentiment, Length: 41157, dtype: int64

In [8]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
X = data["OriginalTweet"]
y = data["Sentiment"]

X_train , X_test , y_train , y_test = train_test_split(X, y, test_size=0.2, random_state=0)
y_train = to_categorical(y_train, 5)
y_test  = to_categorical(y_test, 5)

In [9]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
tokenizer = Tokenizer(num_words = 20000, oov_token="<OOV>")
tokenizer.fit_on_texts(X_train)
list_tokenized_train = tokenizer.texts_to_sequences(X_train)
list_tokenized_test = tokenizer.texts_to_sequences(X_test)

maxlen = 200
embed_size = 128 
padded_train = pad_sequences(list_tokenized_train, maxlen=maxlen)
padded_test = pad_sequences(list_tokenized_test, maxlen=maxlen)

In [13]:
from tensorflow.keras.layers import Dense, Input, LSTM, Embedding, Dropout, Bidirectional
from tensorflow.keras.layers import GlobalMaxPool1D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Sequential
model = Sequential()
model.add(Embedding(20000, embed_size))
model.add(Bidirectional(LSTM(150)))
model.add(Dense(5, activation='softmax'))
adam = Adam(learning_rate=0.01)

#inp = Input(shape=(maxlen, )) 
#x   =  Embedding(20000, embed_size)(inp)
#x   =  LSTM(60, return_sequences=True,name='lstm_layer')(x)
#x   =  GlobalMaxPool1D()(x)
#x   =  Dense(64, activation="relu")(x)
#x   =  Dropout(0.2)(x)
#x   =  Dense(5, activation="softmax")(x)

In [14]:
#from tensorflow.keras.models import Model
#model = Model(inputs=inp, outputs=x)
model.compile(loss='categorical_crossentropy',
              optimizer=adam,
              metrics=['accuracy'])

In [15]:
model.fit(padded_train,y_train, epochs=10,validation_data=(padded_test,y_test))

Epoch 1/10
1029/1029 [==============================] - 99s 88ms/step - loss: 1.0972 - accuracy: 0.5468 - val_loss: 0.8217 - val_accuracy: 0.6933
Epoch 2/10
1029/1029 [==============================] - 90s 87ms/step - loss: 0.6607 - accuracy: 0.7627 - val_loss: 0.7571 - val_accuracy: 0.7256
Epoch 3/10
1029/1029 [==============================] - 90s 87ms/step - loss: 0.5309 - accuracy: 0.8138 - val_loss: 0.7515 - val_accuracy: 0.7337
Epoch 4/10
1029/1029 [==============================] - 90s 87ms/step - loss: 0.4565 - accuracy: 0.8399 - val_loss: 0.8117 - val_accuracy: 0.7267
Epoch 5/10
1029/1029 [==============================] - 90s 87ms/step - loss: 0.4000 - accuracy: 0.8636 - val_loss: 0.8242 - val_accuracy: 0.7353
Epoch 6/10
1029/1029 [==============================] - 90s 87ms/step - loss: 0.3727 - accuracy: 0.8716 - val_loss: 0.8496 - val_accuracy: 0.7283
Epoch 7/10
1029/1029 [==============================] - 90s 87ms/step - loss: 0.3540 - accuracy: 0.8795 - val_loss: 0.8877 -

In [16]:
model.evaluate(padded_test,y_test)

258/258 [==============================] - 10s 37ms/step - loss: 0.9847 - accuracy: 0.7083


[0.9847232699394226, 0.7083333134651184]

In [17]:
model.save('corona_nlp_ver1.h5')